## **Homework 2** Problem 1

In [1]:
%matplotlib inline

### (a) Multilayer Perceptron (MLP)

Generally, when you have to deal with image, text, audio or video data,
you can use standard python packages that load data into a numpy array.
Then you can convert this array into a ``torch.Tensor``.

For vision, PyTorch has a package called ``torchvision``, 
that has data loaders for common datasets such as Imagenet, CIFAR10, MNIST, etc. and data transformers for images, viz., ``torchvision.datasets`` 
and ``torch.utils.data.DataLoader``.
This provides a huge convenience and avoids writing boilerplate code.



Load MNIST dataset using `torchvision.datasets`.

In [2]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

mnist_trainset = ...  # TODO: use built-in functions to load MNIST dataset
mnist_testset = ...   # TODO: use built-in functions to load MNIST dataset
print("Print the training dataset:\n ", mnist_trainset)
print("Print the testing dataset:\n ", mnist_testset)

ModuleNotFoundError: No module named 'torch'

Visualize dataset.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm

fig, axs = plt.subplots(5, 5, figsize = (12, 12))
plt.gray()

# loop through subplots and add mnist images
for i, ax in enumerate(axs.flat):
  ax.imshow(mnist_trainset[i][0][0], cmap=cm.gray_r)
  ax.axis('off')
  ax.set_title('Number {}'.format(mnist_trainset[i][1]))

Create dataloader for neural network.

In [ ]:
train_loader = ...  # TODO: create dataloader from dataset
test_loader = ...   # TODO: create dataloader from dataset
print(train_loader)
print(test_loader)

Iterate through the dataloader.

In [ ]:
for batch_id, (data, label) in enumerate(train_loader):
  print('batch_id:', batch_id)
  print(data.shape)
  break

Follow the above instructions on MNIST dataset, write code to load and visualize the CIFAR-10 dataset in the similar manner.

CIFAR-10 dataset has the classes (listed below): ‘airplane’, ‘automobile’, ‘bird’, ‘cat’, ‘deer’, ‘dog’, ‘frog’, ‘horse’, ‘ship’, ‘truck’. The images in CIFAR-10 are of size $3 \times 32 \times 32$, i.e. 3-channel color images of $32 \times 32$ pixels in size. 

<h4>Note:</h4>
The output of torchvision datasets are PILImage images of range [0, 1].
We transform them to Tensors of normalized range [-1, 1].

In [ ]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 
           'ship', 'truck')
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# TODO: load the CIFAR-10 dataset and build dataloader


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

def cifar_imshow(img):
  img = img / 2 + 0.5     # unnormalize
  npimg = img.numpy()
  return np.transpose(npimg, (1, 2, 0))

# TODO: visualize some samples in the CIFAR-10 dataset


Define an MLP neural network to do classification on CIFAR-10 dataset. 
To feed the data into the MLP, each image with dim of $3 \times 32 \times 32$ needs to be flatten into a vector with dim of $3072$.

Just like how you transfer a Tensor onto the GPU, you transfer the neural net onto the GPU. Let's first define our device as the first visible cuda device if we have CUDA available.

**Note**: remember to keep the logs of training the different models.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
import torch.nn as nn

class MLP(nn.Module):
  def __init__(self):
    super(MLP, self).__init__()
    # TODO: define your MLP

  def forward(self, x):
    # TODO: define your forward function
    return x

mlp = MLP().to(device)  # operate on GPU

Define a loss function and optimizer.

In [ ]:
import torch.optim as optim

# TODO: you can change loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

Train the network. 

This is when things start to get interesting.
We simply have to loop over our data iterator, and feed the inputs to the
network and optimize.

In [ ]:
n_epoch = 5

for epoch in range(n_epoch):  # loop over the dataset multiple times
  for i, data in enumerate(train_loader, 0):
    # TODO: write training code
    pass

Save the trained model. 
See [here](https://pytorch.org/docs/stable/notes/serialization.html)
for more details on saving PyTorch models.

In [ ]:
PATH = './mlp_cifar10.pth'
torch.save(mlp.state_dict(), PATH)

Load back in our saved model. (Note: saving and re-loading the model
wasn't necessary here, we only did it to illustrate how to do so)



In [ ]:
mlp = MLP().to(device)
mlp.load_state_dict(torch.load(PATH))

Evaluate the classfication performance on the testing set.

In [ ]:
with torch.no_grad():
  for data in test_loader:
    # TODO: write testing code
    pass

### (c) Convolution Neural Network (CNN)

Define a CNN to do classification on CIFAR-10 dataset. You can build a CNN from the previous PyTorch tutorial code and modify it to take 3-channel images (instead of 1-channel images as it was defined).

**Note**: remember to keep the logs of training the different models.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    # TODO: define your CNN

  def forward(self, x):
    # TODO: define your forward function  
    return x

cnn = CNN().to(device)  # operate on GPU

Define a loss function and optimizer.

In [ ]:
import torch.optim as optim

# TODO: you can change loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

Train the network.

In [ ]:
n_epoch = 5

for epoch in range(n_epoch):  # loop over the dataset multiple times
  for i, data in enumerate(train_loader, 0):
    # TODO: write training code
    pass

Save the trained model. (Do not upload your trained model to Canvas.)

In [ ]:
PATH = './weights/cnn_cifar10.pth'
torch.save(cnn.state_dict(), PATH)

Evaluate the classfication performance on the testing set.

### (d) Discussions

Write your answers here.